In [1]:
import numpy as np
import os
import sys
import torch
import pytorch3d

import matplotlib.pyplot as plt

import trimesh
from pytorch3d.io import load_objs_as_meshes, save_obj
import numpy as np
from tqdm.notebook import tqdm

#from src.cleansed_cube import Cube

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
from pytorch3d.io import load_objs_as_meshes, save_obj

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader,
    TexturesVertex,
)

from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from src.plot_image_grid import image_grid

In [2]:
import math
import torch
import torch.nn  as nn
import torch.nn.functional as F


from src.operators import get_gaussian

from src.cleansed_cube import SourceCube, sides_dict
from src.discrete_laplacian import DiscreteLaplacian
from src.discrete_gaussian import DiscreteGaussian
from src.padding import pad_side


class ProgressiveCube(nn.Module):
    def __init__(self, ns, kernel=3, sigma=1, clip=None, start=-0.5, end=0.5):
        super(ProgressiveCube, self).__init__()        
        self.ns = ns
        self.n = ns[-1]
        self.kernel= kernel
        self.side_names = list(sides_dict(1).keys())
        self.params = nn.ModuleList([sides_dict(n) for n in ns])
        
        self.source = SourceCube(self.n, start, end)
        #self.gaussian = get_gaussian(kernel)
        self.gaussian = DiscreteGaussian(kernel, sigma=sigma,  padding=False)
        self.laplacian = DiscreteLaplacian()
        clip = clip or 1. / self.n
        for d in self.params:
            for p in d.values():                
                p.register_hook(lambda grad:
                    torch.clamp(torch.nan_to_num(grad), -clip, clip))

    def make_vert(self):
        return torch.cat([p[0].reshape(3, -1).t()
                          for p in self.params.values()])

    def scale(self, t):
        return  F.interpolate(t, self.n, mode='bilinear', align_corners=True)

    def forward(self):
        summed = {}
        for d in self.params:            
            for key in self.side_names:
                if key in summed:
                    summed[key] = summed[key] + self.scale(d[key])
                else:
                    summed[key] = self.scale(d[key])        
        ps = torch.cat([p for p in summed.values()])        
        deform_verts = ps.permute(0, 2, 3, 1).reshape(-1, 3)         
        new_src_mesh = self.source(deform_verts)        
        return new_src_mesh, 0#self.laplacian(ps)    
    
    def smooth(self):
        for i in range(len(self.params)):
            params, sides = self.params[i], {}
            for side_name in params:
                grad = params[side_name].grad[0]        
                sides[side_name] = grad.permute(1, 2, 0)

            for side_name in params:
                padded = pad_side(sides, side_name, self.kernel)
                padded = padded.permute(2, 0, 1)[None]
                padded = self.gaussian(padded)
                self.params[i][side_name].grad.copy_(padded)

    def export(self, f):        
        mesh, _ = self.forward()
        vertices = mesh.vertices[0].cpu().detach()
        faces = mesh.faces.cpu().detach()        
        mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
        mesh.export(f)

cube =  ProgressiveCube([2, 8, 16])
cube.forward()

3


(Mesh(vertices=tensor([[[ 0.5625, -0.5000, -0.5000,  1.0000],
          [ 0.5625, -0.5000, -0.4333,  1.0000],
          [ 0.5625, -0.5000, -0.3667,  1.0000],
          ...,
          [ 0.5000,  0.3667, -0.5625,  1.0000],
          [ 0.5000,  0.4333, -0.5625,  1.0000],
          [ 0.5000,  0.5000, -0.5625,  1.0000]]], grad_fn=<UnsqueezeBackward0>), faces=tensor([[  16,    0,   17],
         [   1,   17,    0],
         [  17,    1,   18],
         ...,
         [1292, 1293,  720],
         [1293, 1294,  736],
         [1294, 1295,  752]], dtype=torch.int32), colors=tensor([[0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000],
         ...,
         [0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000]])),
 0)

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


obj_filename = "./data/bunny.obj"
obj_filename = "./data/nefertiti.obj"
bunny = load_objs_as_meshes([obj_filename], device=device)

textures=TexturesVertex(verts_features=[torch.ones_like(bunny.verts_packed())])

bunny = Meshes(verts=[bunny.verts_packed()], 
           faces=[bunny.faces_packed()],
           textures=textures)

verts = bunny.verts_packed()
N = verts.shape[0]
center = verts.mean(0)
scale = max((verts - center).abs().max(0)[0])
bunny.offset_verts_(-center)
bunny.scale_verts_((1.0 / float(scale)));

In [6]:
n, kernel, sigma = 128, 7, 2

cube = ProgressiveCube([4, 16, 32], kernel=kernel, sigma=sigma, clip=1.).to(device)

optimizer = torch.optim.SGD(cube.parameters(), lr=1., momentum=0.)
#optimizer = torch.optim.Adam(cube.parameters(), lr=0.001)
optimizer

7


SGD (
Parameter Group 0
    dampening: 0
    lr: 1.0
    momentum: 0.0
    nesterov: False
    weight_decay: 0
)

In [7]:
cube.export('./cube32.obj')

In [6]:
Niter = 20001
num_views_per_iteration = 2

loop = tqdm(range(Niter))

laplace_weight =  1.

w_chamfer = 1.0
w_laplacian = 0.1 
w_normal = 0.01 
w_edge = 1.0 

for i in loop:
    stop = False
    
    optimizer.zero_grad()
    
    new_src_mesh, laplace_loss = cube.forward()        
    verts=new_src_mesh.vertices[:, :, :3]
    textures = TexturesVertex(verts_features=torch.ones_like(verts))
    p3d_mesh = Meshes(verts=verts,
                  faces=new_src_mesh.faces[None],
                  textures=textures)
    
    sample_src = sample_points_from_meshes(p3d_mesh, 5000)
    sample_trg = sample_points_from_meshes(bunny, 5000)
    
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    loss_laplacian = mesh_laplacian_smoothing(p3d_mesh, method="uniform")
    loss_normal = mesh_normal_consistency(p3d_mesh)
    loss_edge = mesh_edge_loss(p3d_mesh)
    
    
    
    
    #laplacian_smoothing = mesh_laplacian_smoothing(new_src_mesh, method="uniform")
    sum_loss = torch.tensor(0.0, device=device) 
    
    sum_loss += loss_chamfer * w_chamfer
    #sum_loss += laplace_loss * w_laplacian
    sum_loss += loss_laplacian * w_laplacian
    sum_loss += loss_normal * w_normal
    sum_loss += loss_edge * w_edge
    
    loop.set_description("total_loss = %.6f" % sum_loss)
    
    sum_loss.backward()
    #cube.smooth()
    optimizer.step()
    
    if i % 500 == 0:
        f = f'./data/cube_mesh_{n}_{i}.obj'
        cube.export(f)
        print(f)
#kernel = 0
f = f'./data/cube_mesh_{n}_{kernel}.obj'
cube.export(f)
f

/home/bobi/miniconda3/envs/pytorch3d_06/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630815121/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


./data/cube_mesh_128_0.obj
./data/cube_mesh_128_500.obj
./data/cube_mesh_128_1000.obj
./data/cube_mesh_128_1500.obj
./data/cube_mesh_128_2000.obj
./data/cube_mesh_128_2500.obj
./data/cube_mesh_128_3000.obj
./data/cube_mesh_128_3500.obj
./data/cube_mesh_128_4000.obj
./data/cube_mesh_128_4500.obj
./data/cube_mesh_128_5000.obj
./data/cube_mesh_128_5500.obj
./data/cube_mesh_128_6000.obj
./data/cube_mesh_128_6500.obj
./data/cube_mesh_128_7000.obj
./data/cube_mesh_128_7500.obj
./data/cube_mesh_128_8000.obj
./data/cube_mesh_128_8500.obj
./data/cube_mesh_128_9000.obj
./data/cube_mesh_128_9500.obj
./data/cube_mesh_128_10000.obj
./data/cube_mesh_128_10500.obj
./data/cube_mesh_128_11000.obj
./data/cube_mesh_128_11500.obj
./data/cube_mesh_128_12000.obj
./data/cube_mesh_128_12500.obj
./data/cube_mesh_128_13000.obj
./data/cube_mesh_128_13500.obj
./data/cube_mesh_128_14000.obj
./data/cube_mesh_128_14500.obj
./data/cube_mesh_128_15000.obj
./data/cube_mesh_128_15500.obj
./data/cube_mesh_128_16000.obj
.

'./data/cube_mesh_128_7.obj'

In [2]:
for i in range(1, 10):
    print(i, 2**i)

1 2
2 4
3 8
4 16
5 32
6 64
7 128
8 256
9 512
